In [1]:
import sys
import gym
#import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential

from scores.score_logger import ScoreLogger

EPISODES = 300

/Users/aashishkumar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/aashishkumar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Double DQN Agent for the Cartpole
# it uses Neural Network to approximate q function
# and replay memory & target q network
class DoubleDQNAgent:
    def __init__(self, state_size, action_size):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # these is hyper parameters for the Double DQN
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.batch_size = 64
        self.train_start = 1000
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()
        self.target_model = self.build_model()

        # initialize target model
        self.update_target_model()

        if self.load_model:
            self.model.load_weights("./save_model/cartpole_ddqn.h5")

    # approximate Q function using Neural Network
    # state is input and Q Value of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(24, activation='relu',
                        kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear',
                        kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    # after some time interval update the target model to be same with model
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # get action from model using epsilon-greedy policy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    # save sample <s,a,r,s'> to the replay memory
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    # pick samples randomly from replay memory (with batch_size)
    def train_model(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(batch_size):
            update_input[i] = mini_batch[i][0]
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3]
            done.append(mini_batch[i][4])

        target = self.model.predict(update_input)
        target_next = self.model.predict(update_target)
        target_val = self.target_model.predict(update_target)

        for i in range(self.batch_size):
            # like Q Learning, get maximum Q value at s'
            # But from target model
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                # the key point of Double DQN
                # selection of action is from model
                # update is from target model
                a = np.argmax(target_next[i])
                target[i][action[i]] = reward[i] + self.discount_factor * (
                    target_val[i][a])

        # make minibatch which includes target q value and predicted q value
        # and do the model fit!
        self.model.fit(update_input, target, batch_size=self.batch_size,
                       epochs=1, verbose=0)



In [ ]:
ENV_NAME = "CartPole-v1"
env = gym.make(ENV_NAME)
# get size of state and action from environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

score_logger = ScoreLogger(ENV_NAME)

agent = DoubleDQNAgent(state_size, action_size)

scores, episodes = [], []
run = 0

for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    step = 0
    run += 1
    while not done:
        step += 1
        if agent.render:
            env.render()

        # get action for the current state and go one step in environment
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
        #reward = reward if not done or score == 499 else -100
        reward = reward if not done else -reward

        # save the sample <s, a, r, s'> to the replay memory
        agent.append_sample(state, action, reward, next_state, done)
        # every time step do the training
        agent.train_model()
        score += reward
        state = next_state

        if done:
            # every episode update the target model to be same with model
            agent.update_target_model()

            # every episode, plot the play time
            #score = score if score == 500 else score + 100
            #scores.append(score)
            
            print("Run: " + str(run) + ", exploration: " + str(agent.epsilon) + ", score: " + str(step))
            score_logger.add_score(step, run)
                
#             episodes.append(e)
#             pylab.plot(episodes, scores, 'b')
#             pylab.savefig("./save_graph/cartpole_ddqn.png")
#             print("episode:", e, "  score:", score, "  memory length:",
#                   len(agent.memory), "  epsilon:", agent.epsilon)

            # if the mean of scores of last 10 episode is bigger than 490
            # stop training
#             if np.mean(scores[-min(10, len(scores)):]) > 490:
#                 sys.exit()

    # save the model
    # if e % 50 == 0:
    #     agent.model.save_weights("./save_model/cartpole_ddqn.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_5 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_6 (Den

/Users/aashishkumar/Documents/Projects/Robust-RL/scores/score_logger.py:77: RankWarning: Polyfit may be poorly conditioned
  z = np.polyfit(np.array(trend_x), np.array(y[1:]), 1)



Scores: (min: 14, avg: 15.666666666666666, max: 19)

Run: 4, exploration: 0.9407945259609451, score: 14
Scores: (min: 14, avg: 15.25, max: 19)

Run: 5, exploration: 0.9138900318559524, score: 29
Scores: (min: 14, avg: 18, max: 29)

Run: 6, exploration: 0.8948890480710096, score: 21
Scores: (min: 14, avg: 18.5, max: 29)

Run: 7, exploration: 0.8754068367770318, score: 22
Scores: (min: 14, avg: 19, max: 29)

Run: 8, exploration: 0.847823709077432, score: 32
Scores: (min: 14, avg: 20.625, max: 32)

Run: 9, exploration: 0.8301963316171974, score: 21
Scores: (min: 14, avg: 20.666666666666668, max: 32)

Run: 10, exploration: 0.8088788946494789, score: 26
Scores: (min: 14, avg: 21.2, max: 32)

Run: 11, exploration: 0.8000255819490795, score: 11
Scores: (min: 11, avg: 20.272727272727273, max: 32)

Run: 12, exploration: 0.7920612314455105, score: 10
Scores: (min: 10, avg: 19.416666666666668, max: 32)

Run: 13, exploration: 0.7771467460721305, score: 19
Scores: (min: 10, avg: 19.384615384615383

Run: 85, exploration: 0.26536117873480936, score: 16
Scores: (min: 8, avg: 15.6, max: 44)

Run: 86, exploration: 0.262719476412822, score: 10
Scores: (min: 8, avg: 15.534883720930232, max: 44)

Run: 87, exploration: 0.260364436990891, score: 9
Scores: (min: 8, avg: 15.459770114942529, max: 44)

Run: 88, exploration: 0.2577724778315252, score: 10
Scores: (min: 8, avg: 15.397727272727273, max: 44)

Run: 89, exploration: 0.25495111561414624, score: 11
Scores: (min: 8, avg: 15.348314606741573, max: 44)

Run: 90, exploration: 0.2526657124299791, score: 9
Scores: (min: 8, avg: 15.277777777777779, max: 44)

Run: 91, exploration: 0.25065144723887983, score: 8
Scores: (min: 8, avg: 15.197802197802197, max: 44)

Run: 92, exploration: 0.2481561820560168, score: 10
Scores: (min: 8, avg: 15.141304347826088, max: 44)

Run: 93, exploration: 0.24421532336806764, score: 16
Scores: (min: 8, avg: 15.150537634408602, max: 44)

Run: 94, exploration: 0.23604755704357036, score: 34
Scores: (min: 8, avg: 15.3

Run: 170, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 187.11, max: 500)

Run: 171, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 192, max: 500)

Run: 172, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 196.89, max: 500)

Solved in 72 runs, 172 total runs.
Run: 173, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 201.8, max: 500)

Solved in 73 runs, 173 total runs.
Run: 174, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 206.7, max: 500)

Solved in 74 runs, 174 total runs.
Run: 175, exploration: 0.009998671593271896, score: 500
Scores: (min: 8, avg: 211.59, max: 500)

Solved in 75 runs, 175 total runs.


In [4]:
agent.model.save_weights("my_model_no_noisy_reward.h5")